In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

# Load the Oxford-IIIT Pet dataset
(ds_train, ds_test), ds_info = tfds.load('oxford_iiit_pet', split=['train', 'test'], as_supervised=True, with_info=True)

# Define a function to resize images
def resize_image(image, label):
    image = tf.image.resize(image, (224, 224))
    return image, label

# Apply resizing to the dataset
ds_train = ds_train.map(resize_image)
ds_test = ds_test.map(resize_image)

# Normalize pixel values to be in the range [0, 1]
ds_train = ds_train.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, y))
ds_test = ds_test.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, y))

# Shuffle and batch the datasets
ds_train = ds_train.shuffle(10000).batch(64)
ds_test = ds_test.batch(64)

# Convert labels to categorical format
def categorical_labels(images, labels):
    return images, tf.one_hot(labels, depth=37)  # 37 classes in Oxford-IIIT Pet dataset

ds_train = ds_train.map(categorical_labels)
ds_test = ds_test.map(categorical_labels)

# Verify the shapes
for images, labels in ds_train.take(1):
    print("Images shape:", images.shape)
    print("Labels shape:", labels.shape)

for images, labels in ds_test.take(1):
    print("Images shape:", images.shape)
    print("Labels shape:", labels.shape)


Images shape: (64, 224, 224, 3)
Labels shape: (64, 37)
Images shape: (64, 224, 224, 3)
Labels shape: (64, 37)


In [2]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from PIL import Image
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical


# Create the ResNet model without the top (fully connected) layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    
# Add the top layers for classification
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(37, activation='softmax'))  # Change the number of units and activation
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(ds_train, batch_size=64, epochs=50)

Epoch 1/50
58/58 [==============================] - 1172s 20s/step - loss: 1.8469 - accuracy: 0.4745
Epoch 2/50
58/58 [==============================] - 1167s 20s/step - loss: 0.7942 - accuracy: 0.7522
Epoch 3/50
58/58 [==============================] - 1176s 20s/step - loss: 0.3935 - accuracy: 0.8726
Epoch 4/50
58/58 [==============================] - 1165s 20s/step - loss: 0.2741 - accuracy: 0.9087
Epoch 5/50
58/58 [==============================] - 1187s 20s/step - loss: 0.1940 - accuracy: 0.9391
Epoch 6/50
58/58 [==============================] - 1288s 22s/step - loss: 0.2136 - accuracy: 0.9315
Epoch 7/50
58/58 [==============================] - 1283s 22s/step - loss: 0.1444 - accuracy: 0.9541
Epoch 8/50
58/58 [==============================] - 1317s 22s/step - loss: 0.0989 - accuracy: 0.9668
Epoch 9/50
58/58 [==============================] - 1387s 24s/step - loss: 0.1231 - accuracy: 0.9636
Epoch 10/50
58/58 [==============================] - 1389s 24s/step - loss: 0.1324 - accura

In [3]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

# Make predictions on the test dataset
predictions = model.predict(ds_test)

# Convert one-hot encoded predictions back to class labels
predicted_labels = np.argmax(predictions, axis=1)

# Manually generate ground truth labels for evaluation
true_labels = []
for _, label in ds_test:
    true_labels.extend(label.numpy())

true_labels = np.argmax(true_labels, axis=1)

# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)

# Calculate F1 score
f1 = f1_score(true_labels, predicted_labels, average='weighted')

# Calculate confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)

# Assuming you have test_loss and test_accuracy available
# If not, you'll need to calculate them based on your model and test dataset
print(f"Test Accuracy: {accuracy}")
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")


Test Accuracy: 0.7130008176614882
Accuracy: 0.7130008176614882
F1 Score: 0.710499994201091
